# Face recognition modeling

In the following notebook, I will fit a variety of models to do image classification.

**Selected models**:
* Convolutional Neural Network (CNN),
* Hugging face image classification transformer,
* ViTs

All of them will be fit in order to compare their performance and offer benchmarks.

**Hyperparameter tuning**:
* steps per epoch,
* epoch,
* learning rate,
* and other parameters

Will be fit using RandomSearch (NOT GridSearch :) )

In [12]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras import models, layers
import tqdm
from PIL import Image

from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

import os

from google.cloud import storage
import numpy as np

In [16]:
def read_arrays_from_gcs(bucket_name, imgs_filename, lbls_filename):
    """
    Reads the imgs and lbls arrays from Google Cloud Storage (GCS).

    Args:
        bucket_name (str): The name of the GCS bucket.
        imgs_filename (str): The filename for the imgs array.
        lbls_filename (str): The filename for the lbls array.

    Returns:
        tuple: A tuple containing the imgs and lbls arrays.
    """
    # Set up the GCS client
    client = storage.Client()

    # Read the .npy files from the GCS bucket
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(imgs_filename)
    blob.download_to_filename(imgs_filename)
    blob = bucket.blob(lbls_filename)
    blob.download_to_filename(lbls_filename)

    # Load the .npy files into NumPy arrays
    imgs = np.load(imgs_filename)
    lbls = np.load(lbls_filename)

    return imgs, lbls

In [17]:
bucket_name = "msca-ml-final-project"
train_imgs_filename = "train_imgs.npy"
train_lbls_filename = "train_lbls.npy"
test_imgs_filename = "test_imgs.npy"
test_lbls_filename = "test_lbls.npy"
val_imgs_filename = "val_imgs.npy"
val_lbls_filename = "val_lbls.npy"

train_imgs, train_lbls = read_arrays_from_gcs(bucket_name, train_imgs_filename, train_lbls_filename)
test_imgs, test_lbls = read_arrays_from_gcs(bucket_name, test_imgs_filename, test_lbls_filename)
val_imgs, val_lbls = read_arrays_from_gcs(bucket_name, val_imgs_filename, val_lbls_filename)

Print shape of all files to make sure they are correct

In [18]:
print(f'Shape of training data: {np.shape(train_imgs)}')
print(f'Shape of testing data: {np.shape(test_imgs)}')
print(f'Shape of validation data: {np.shape(val_imgs)}')

Shape of training data: (28709, 48, 48, 1)
Shape of testing data: (3589, 48, 48, 1)
Shape of validation data: (3589, 48, 48, 1)


## Model fitting

#### 1. CNN - LeNet architecture

In [6]:
# defining LeNet architecture
def create_lenet_model(input_shape, num_classes):
    model_lenet = Sequential()
    
    # Convolutional layers
    model_lenet.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    model_lenet.add(AveragePooling2D(pool_size=(2, 2)))
    model_lenet.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
    model_lenet.add(AveragePooling2D(pool_size=(2, 2)))
    
    # Flatten the 3D output to 1D
    model_lenet.add(Flatten())
    
    # Fully connected layers
    model_lenet.add(Dense(120, activation='relu'))
    model_lenet.add(Dense(84, activation='relu'))
    
    # Output layer
    model_lenet.add(Dense(num_classes, activation='softmax'))
    
    return model_lenet

# Create an instance of the LeNet model
input_shape = (48, 48, 1)  # Input shape of your images
num_classes = 7  # Number of classes for classification
model_lenet = create_lenet_model(input_shape, num_classes)

# Compile the model
model_lenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model_lenet.summary()

2023-05-15 23:55:09.694072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-15 23:55:09.694128: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-15 23:55:09.694156: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-ml-project): /proc/driver/nvidia/version does not exist
2023-05-15 23:55:09.695770: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 44, 44, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 22, 22, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 18, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 9, 9, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 1296)              0         
                                                                 
 dense (Dense)               (None, 120)               1

In [ ]:
# Training the model, and validating
model_lenet.fit(train_imgs, train_lbls, 
          epochs=5, batch_size=32, 
          validation_data=(val_imgs, val_lbls), verbose=1)